# Scrape Property Tax Data (pre-2018)

Using NYC Dept of Finance website, download property tax data for all buildings in Assembly District 36, under the 421A Tax Abatement program. 

Each Tax Class has a different property tax form so we will pull each tax class individually.

Tax classes are:
- 1
- 2
- 2A
- 2B


In [23]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from urllib.request import urlretrieve
import json
import tabula as tb
from datetime import datetime
import re

## Tax Class 2

### Read Building Data

In [24]:
df_buildings = pd.read_csv("../data/created_data/ad36_421Properties_TaxClass2.csv")
df_buildings = df_buildings.astype({'BBL': 'str'})

In [25]:
print("Total Number of buildings under this class: {}".format(df_buildings.shape[0]))
df_buildings.head()

Total Number of buildings under this class: 71


,Unnamed: 0,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,...,BUILDING CLASS,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,BBL
0,0,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4006371,2009.0,2017-08-22T00:00:00.000Z,Constructed,1113607,100.194260,2100,10.0,...,D6,21-16 31 AVENUE,11106.0,32,1,33,8550,33500,2009,4005520022
1,7,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4005656,2008.0,2017-08-22T00:00:00.000Z,Constructed,1109509,53.890919,2100,10.0,...,D1,12-26 30 AVENUE,11102.0,37,0,37,13575,39745,2008,4005150031
2,11,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4005897,2010.0,2017-08-22T00:00:00.000Z,Constructed,1259711,63.000000,2100,11.0,...,D1,14-34 31 AVENUE,11106.0,14,0,14,4726,12945,2010,4005330037
3,14,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4019881,2008.0,2017-08-22T00:00:00.000Z,Constructed,1108121,59.000000,2100,30.0,...,D1,18-05 27 AVENUE,11102.0,11,0,11,2640,15900,2008,4008850016
4,15,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4540411,2012.0,2017-08-22T00:00:00.000Z,Constructed,1256175,64.000000,2100,15.0,...,D1,23-12 30 DRIVE,11102.0,20,0,20,7512,16000,2012,4005700030


In [26]:
#get BBL values into a list
building_bbl_list = list(df_buildings['BBL'])

### Extract Property Data

For each BBL, we need to download the property tax PDF, then gather the requisite 421 data and save it into a list of Series(?). 

Eventually, combine into a single dataframe and merge with the buildings dataframe above. 

Link to download PDF: https://a836-edms.nyc.gov/dctm-rest/repositories/dofedmspts/StatementSearch?bbl=[BBL VALUE HERE]&stmtDate=20211120&stmtType=SOA 

In [30]:
#list to include all the rent stabilization data
property_tax_list = []

In [31]:
tax_dates = ['20180601', '20170602', '20160603']

In [32]:
#Full trial run

# tax_form_date = '20211120'

for tax_form_date in tax_dates:


    for i in range(5):
    # for i in range(len(building_bbl_list)):

        print("BBL: {} Tax Date: {}".format(building_bbl_list[i], tax_form_date))

        #Download PDF 
        url_loc = "https://a836-mspuvw-dofptsz.nyc.gov/PTSCM/StatementSearch?bbl={}&stmtDate={}&stmtType=SOA".format(building_bbl_list[i], tax_form_date)

        urlretrieve(url_loc, "property_tax.pdf")

        #read second page into a dataframe
        try:
            file = 'property_tax.pdf'
            df_pdf= tb.read_pdf(file, pages = '2', area = (50, 0, 1000, 1000), columns = [200, 305, 365, 455, 500])[0]
            #Find Rent data and save into a new dataframe
            try:
                # index_to_find = df_pdf.loc[df_pdf['Unnamed: 0'] == 'Rent Stabilization'].index
                # df_pdf.columns = df_pdf.iloc[index_to_find[0]]
                # # df_pdf = df_pdf.loc[df_pdf['Rent Stabilization']]
                # df_pdf = df_pdf.iloc[index_to_find[0]+1]
                # df_list = df_pdf.to_list()
                # df_list.append(str(building_bbl_list[i]))
                # df_list.append(tax_form_date)
                # final_df = pd.DataFrame(df_pdf).T
                index_to_find = df_pdf.loc[df_pdf['Unnamed: 1'] == '# Apts'].index
                df_pdf.columns = df_pdf.iloc[index_to_find[0]]
                df_pdf = df_pdf.iloc[index_to_find[0]+1]
                df_list = df_pdf.to_list()
                df_list.append(str(building_bbl_list[i]))
                df_list.append(tax_form_date)
            except IndexError:
                df_list = ['Rent Stabilization Fee - Chg', '0', 'NaN', 'nan', 'nan', '0', str(building_bbl_list[i]), tax_form_date]
                print("No Rent Stabilized Units")
        except:
            print("Error in Tabula")



        


        property_tax_list.append(df_list)
cols = ['Rent Stabilized', 'Units', 'Due Date', 'RS ID', 'To Drop', 'Total Charge', 'BBL', 'Date']
final_df = pd.DataFrame(property_tax_list, columns=cols)
# final_df['Date'] = tax_form_date


BBL: 4005520022 Tax Date: 20180601


Error from tabula-java:
Error: Error: End-of-File, expected line




Error in Tabula
BBL: 4005150031 Tax Date: 20180601


Error from tabula-java:
Error: Error: End-of-File, expected line




Error in Tabula
BBL: 4005330037 Tax Date: 20180601


...

Error in Tabula
BBL: 4008850016 Tax Date: 20160603


Error from tabula-java:
Error: Error: End-of-File, expected line




Error in Tabula
BBL: 4005700030 Tax Date: 20160603


Error from tabula-java:
Error: Error: End-of-File, expected line




Error in Tabula


In [ ]:
#merge with building data
merged_data = df_buildings.merge(final_df, left_on='BBL', right_on='BBL', how = 'inner')

In [ ]:
merged_data

,Unnamed: 0,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,...,GROSS SQUARE FEET,YEAR BUILT,BBL,Rent Stabilized,Units,Due Date,RS ID,To Drop,Total Charge,Date
0,0,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4006371,2009.0,2017-08-22T00:00:00.000Z,Constructed,1113607,100.194260,2100,10.0,...,33500,2009,4005520022,Housing-Rent Stabilization,30,01/01/2019,041089700 08970,0,$300.00,20180601
1,7,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4005656,2008.0,2017-08-22T00:00:00.000Z,Constructed,1109509,53.890919,2100,10.0,...,39745,2008,4005150031,Rent Stabilization Fee - Chg,0,NaN,nan,nan,0,20180601
2,11,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4005897,2010.0,2017-08-22T00:00:00.000Z,Constructed,1259711,63.000000,2100,11.0,...,12945,2010,4005330037,Housing-Rent Stabilization,14,01/01/2019,043176000 17600,0,$140.00,20180601
3,14,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4019881,2008.0,2017-08-22T00:00:00.000Z,Constructed,1108121,59.000000,2100,30.0,...,15900,2008,4008850016,Housing-Rent Stabilization,11,01/01/2019,042862100 86210,0,$110.00,20180601
4,15,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4540411,2012.0,2017-08-22T00:00:00.000Z,Constructed,1256175,64.000000,2100,15.0,...,16000,2012,4005700030,Housing-Rent Stabilization,20,01/01/2019,043092400 09240,0,$200.00,20180601
5,16,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4616243,NaN,2017-09-14T00:00:00.000Z,Constructed,1268319,10.033600,5110,15.0,...,16000,2012,4005700030,Housing-Rent Stabilization,20,01/01/2019,043092400 09240,0,$200.00,20180601


In [ ]:
merged_data['Date'] = merged_data['Date'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))
merged_data['Year'] = merged_data['Date'].apply(lambda x: x.year)

In [ ]:
df_class2 = merged_data[['the_geom', 'geomsource', 'cnstrct_yr', 'lstmoddate', 'lststatype', 'Latitude', 'Longitude',
                         'AssemDist', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS', 'ADDRESS', 'ZIP CODE', 'RESIDENTIAL UNITS',
                         'COMMERCIAL UNITS', 'TOTAL UNITS', 'YEAR BUILT', 'BBL', 'Units', 'Total Charge', 'Date', 'Year']]
df_class2.rename(columns={'Units': 'Rent Stabilized Units', 'Total Charge': 'Rent Stabilized Charges', 'Date': 'Tax Doc Date',
                                'Year': 'Tax Doc Year'}, inplace = True)
df_class2.fillna(0, inplace = True)
df_class2['cnstrct_yr'] = df_class2['cnstrct_yr'].astype(int)
df_pivot2 = df_class2.drop(['the_geom'], axis = 1)
df_pivot2 = df_pivot2.sort_values('geomsource', ascending=False).drop_duplicates(subset=['ADDRESS', 'Tax Doc Year'])
df_pivot2 = df_pivot2.pivot(index=['ADDRESS', 'Latitude', 'Longitude'], columns='Tax Doc Year', values='Rent Stabilized Units')
df_pivot2.fillna(0, inplace=True)
df_pivot2.to_csv('../data/created_data/taxClass2_pivotTable_ad36_pre2019.csv')

/Users/rohun/opt/miniconda3/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/rohun/opt/miniconda3/lib/python3.9/site-packages/pandas/core/frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-21-ccf36bde1964>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [22]:
merged_data.to_csv('../data/created_data/ad36_421Properties_TaxClass2_withTaxData_pre2019.csv')

## Tax Class 2B

### Read Building Data

In [22]:
df_buildings = pd.read_csv("ad36_421Properties_TaxClass2B.csv")
df_buildings = df_buildings.astype({'BBL': 'str'})

In [23]:
print("Total Number of buildings under this class: {}".format(df_buildings.shape[0]))
df_buildings.head(1)

Total Number of buildings under this class: 105


,Unnamed: 0,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,...,BUILDING CLASS,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,BBL
0,1,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4594935,2009.0,2017-08-22T00:00:00.000Z,Constructed,1112303,54.768241,2100,35,...,C1,25-16 30 DRIVE,11102.0,8,0,8,3669,10976,2009,4005780026


In [24]:
#get BBL values into a list
building_bbl_list = list(df_buildings['BBL'])

### Extract Property Data

For each BBL, we need to download the property tax PDF, then gather the requisite 421 data and save it into a list of Series(?). 

Eventually, combine into a single dataframe and merge with the buildings dataframe above. 

Link to download PDF: https://a836-edms.nyc.gov/dctm-rest/repositories/dofedmspts/StatementSearch?bbl=[BBL VALUE HERE]&stmtDate=20211120&stmtType=SOA 

In [25]:
#list to include all the rent stabilization data
property_tax_list = []

In [26]:
tax_dates = ['20180601', '20170602', '20160603']

In [27]:
#Full trial run

# tax_form_date = '20211120'

for tax_form_date in tax_dates:



    for i in range(len(building_bbl_list)):

        print("BBL: {} Tax Date: {}".format(building_bbl_list[i], tax_form_date))

        #Download PDF 
        url_loc = "https://a836-mspuvw-dofptsz.nyc.gov/PTSCM/StatementSearch?bbl={}&stmtDate={}&stmtType=SOA".format(building_bbl_list[i], tax_form_date)

        urlretrieve(url_loc, "property_tax.pdf")

        #read second page into a dataframe
        try:
            file = 'property_tax.pdf'
            df_pdf= tb.read_pdf(file, pages = '2', area = (50, 0, 1000, 1000), columns = [200, 305, 365, 455, 500])[0]
            #Find Rent data and save into a new dataframe
            try:
                # index_to_find = df_pdf.loc[df_pdf['Unnamed: 0'] == 'Rent Stabilization'].index
                # df_pdf.columns = df_pdf.iloc[index_to_find[0]]
                # # df_pdf = df_pdf.loc[df_pdf['Rent Stabilization']]
                # df_pdf = df_pdf.iloc[index_to_find[0]+1]
                # df_list = df_pdf.to_list()
                # df_list.append(str(building_bbl_list[i]))
                # df_list.append(tax_form_date)
                # final_df = pd.DataFrame(df_pdf).T
                index_to_find = df_pdf.loc[df_pdf['Unnamed: 1'] == '# Apts'].index
                df_pdf.columns = df_pdf.iloc[index_to_find[0]]
                df_pdf = df_pdf.iloc[index_to_find[0]+1]
                df_list = df_pdf.to_list()
                df_list.append(str(building_bbl_list[i]))
                df_list.append(tax_form_date)
            except IndexError:
                df_list = ['Rent Stabilization Fee - Chg', '0', 'NaN', 'nan', 'nan', '0', str(building_bbl_list[i]), tax_form_date]
                print("No Rent Stabilized Units")
        except:
            print("Error in Tabula")



        


        property_tax_list.append(df_list)
cols = ['Rent Stabilized', 'Units', 'Due Date', 'RS ID', 'To Drop', 'Total Charge', 'BBL', 'Date']
final_df = pd.DataFrame(property_tax_list, columns=cols)
# final_df['Date'] = tax_form_date

BBL: 4005780026 Tax Date: 20180601


Error from tabula-java:
Error: Error: End-of-File, expected line




Error in Tabula
BBL: 4006600024 Tax Date: 20180601


Error from tabula-java:
Error: Error: End-of-File, expected line




Error in Tabula
BBL: 4008610044 Tax Date: 20180601


...

Error in Tabula
BBL: 4006190057 Tax Date: 20160603


Error from tabula-java:
Error: Error: End-of-File, expected line




Error in Tabula
BBL: 4005940155 Tax Date: 20160603


Error from tabula-java:
Error: Error: End-of-File, expected line




Error in Tabula


In [28]:
merged_data = df_buildings.merge(final_df, left_on='BBL', right_on='BBL', how = 'inner')

In [29]:
merged_data['Date'] = merged_data['Date'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))
merged_data['Year'] = merged_data['Date'].apply(lambda x: x.year)

In [30]:
df_class2b = merged_data[['the_geom', 'geomsource', 'cnstrct_yr', 'lstmoddate', 'lststatype', 'Latitude', 'Longitude',
                         'AssemDist', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS', 'ADDRESS', 'ZIP CODE', 'RESIDENTIAL UNITS',
                         'COMMERCIAL UNITS', 'TOTAL UNITS', 'YEAR BUILT', 'BBL', 'Units', 'Total Charge', 'Date', 'Year']]
df_class2b.rename(columns={'Units': 'Rent Stabilized Units', 'Total Charge': 'Rent Stabilized Charges', 'Date': 'Tax Doc Date',
                                'Year': 'Tax Doc Year'}, inplace = True)
df_class2b.fillna(0, inplace = True)
df_class2b['cnstrct_yr'] = df_class2b['cnstrct_yr'].astype(int)
df_pivot2b = df_class2b.drop(['the_geom'], axis = 1)
df_pivot2b = df_pivot2b.sort_values('geomsource', ascending=False).drop_duplicates(subset=['ADDRESS', 'Tax Doc Year'])
df_pivot2b = df_pivot2b.pivot(index=['ADDRESS', 'Latitude', 'Longitude'], columns='Tax Doc Year', values='Rent Stabilized Units')
df_pivot2b.fillna(0, inplace = True)
df_pivot2b.to_csv('taxClass2B_pivotTable_ad36_pre2019.csv')

/Users/rohun/opt/miniconda3/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/rohun/opt/miniconda3/lib/python3.9/site-packages/pandas/core/frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [31]:
merged_data.to_csv('ad36_421Properties_TaxClass2B_withTaxData_pre2019.csv')

## Pivot Table

Include this into the actual data above. But for now, read in two CSVs from above and pivot them on the date.

Also in clean up the data. Reduce the number of columns 

Total Charge -> int

In [32]:
# df_class2b = df_class2b[['the_geom', 'geomsource', 'cnstrct_yr', 'lstmoddate', 'lststatype', 'Latitude', 'Longitude',
#                          'AssemDist', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS', 'ADDRESS', 'ZIP CODE', 'RESIDENTIAL UNITS',
#                          'COMMERCIAL UNITS', 'TOTAL UNITS', 'YEAR BUILT', 'BBL', 'Units', 'Total Charge', 'Date', 'Year']]
# df_class2b.rename(columns={'Units': 'Rent Stabilized Units', 'Total Charge': 'Rent Stabilized Charges', 'Date': 'Tax Doc Date',
#                                 'Year': 'Tax Doc Year'}, inplace = True)
# df_class2b.fillna(0, inplace = True)
# df_class2b['cnstrct_yr'] = df_class2b['cnstrct_yr'].astype(int)
# df_pivot2b = df_class2b.drop(['the_geom'], axis = 1)
# df_pivot2b = df_pivot2b.sort_values('geomsource', ascending=False).drop_duplicates(subset=['ADDRESS', 'Tax Doc Year'])
# df_pivot2b = df_pivot2b.pivot(index='ADDRESS', columns='Tax Doc Year', values='Rent Stabilized Units')
# df_pivot2b.fillna(0, inplace = True)
# df_pivot2b.to_csv('taxClass2B_pivotTable_ad36.csv')

In [33]:
# df_class2 = pd.read_csv('ad36_421Properties_TaxClass2B_withTaxData.csv')
# df_class2 = df_class2[['the_geom', 'geomsource', 'cnstrct_yr', 'lstmoddate', 'lststatype', 'Latitude', 'Longitude',
#                          'AssemDist', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS', 'ADDRESS', 'ZIP CODE', 'RESIDENTIAL UNITS',
#                          'COMMERCIAL UNITS', 'TOTAL UNITS', 'YEAR BUILT', 'BBL', 'Units', 'Total Charge', 'Date', 'Year']]
# df_class2.rename(columns={'Units': 'Rent Stabilized Units', 'Total Charge': 'Rent Stabilized Charges', 'Date': 'Tax Doc Date',
#                                 'Year': 'Tax Doc Year'}, inplace = True)
# df_class2.fillna(0, inplace = True)
# df_class2['cnstrct_yr'] = df_class2['cnstrct_yr'].astype(int)
# df_pivot2 = df_class2.drop(['the_geom'], axis = 1)
# df_pivot2 = df_pivot2.sort_values('geomsource', ascending=False).drop_duplicates(subset=['ADDRESS', 'Tax Doc Year'])
# df_pivot2 = df_pivot2.pivot(index=['ADDRESS', 'Latitude', 'Longitude'], columns='Tax Doc Year', values='Rent Stabilized Units')
# df_pivot2.fillna(0, inplace=True)
# # df_pivot2.to_csv('taxClass2_pivotTable_ad36.csv')

## Troubleshooting!


In [34]:
#All this should be done later. Focus on rent stabilization efforts first. 

bbl_trial = '4005780026'
tax_form_date = '20170602'

url_loc = "https://a836-mspuvw-dofptsz.nyc.gov/PTSCM/StatementSearch?bbl={}&stmtDate={}&stmtType=SOA".format(bbl_trial, tax_form_date)

urlretrieve(url_loc, "property_tax.pdf")

('property_tax.pdf', <http.client.HTTPMessage at 0x7fbd288dcfd0>)

In [35]:
file = 'property_tax.pdf'
df3= tb.read_pdf(file, pages = '2', area = (50, 0, 400, 1000), columns = [200, 305, 365, 455, 500])[0]

Error from tabula-java:
Error: Error: End-of-File, expected line




CalledProcessError: Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/rohun/opt/miniconda3/lib/python3.9/site-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar', '--pages', '2', '--area', '50,0,400,1000', '--format', 'JSON', '--columns', '200,305,365,455,500', 'property_tax.pdf']' returned non-zero exit status 1.

In [ ]:
df3

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Page 2
0,Billing Summary,Activity Date,Due Date,NaN,NaN,Amount
1,Outstanding charges including interes,t and payments,NaN,NaN,NaN,$88.84
2,Finance-Property Tax,NaN,07/01/2017,NaN,NaN,$672.96
3,Total amount due,NaN,NaN,NaN,NaN,$761.80
4,Tax Year Charges Remaining,Activity Date,Due Date,NaN,NaN,Amount
5,Finance-Property Tax,NaN,10/01/2017,NaN,NaN,$672.96
6,Finance-Property Tax,NaN,01/01/2018,NaN,NaN,$672.96
7,Finance-Property Tax,NaN,04/01/2018,NaN,NaN,$672.96
8,Total tax year charges remaining,NaN,NaN,NaN,NaN,"$2,018.88"
9,If you pay everything you owe by July,"3, 2017, you would sav",e:,NaN,NaN,$13.46


In [ ]:
index_to_find = df3.loc[df3['Unnamed: 1'] == '# Apts'].index
df3.columns = df3.iloc[index_to_find[0]]
df3 = df3.iloc[index_to_find[0]+1]
df3 = pd.DataFrame(df3).T
df3

IndexError: index 0 is out of bounds for axis 0 with size 0